In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.callbacks import get_openai_callback
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

cache_dir = LocalFileStore("./tmp/embeddings")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100
)

loader = UnstructuredFileLoader("./tmp/read3.pdf")

docs = loader.load_and_split(text_splitter=splitter) 

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

qq = "年間保険料をいくらだと記載されている、被保険者名または対象者はなんと記載されている"
examples = [
{
    "question": qq,
    "answer": """
        年間保険料: 1,000,000円
        被保険者名: 山田太郎
        通貨: 日本円
    """,
},
{
    "question": qq,
    "answer": """
        年間保険料: 1,000ドル
        被保険者名: 山田太郎
        通貨: ドル
    """,
},
{
    "question": qq,
    "answer": """
        年間保険料: 1,000豪ドル
        被保険者名: 山田太郎
        通貨: 豪ドル
    """,
},
{
    "question": qq,
    "answer": """
        年間保険料: 1,000ユーロ
        被保険者名: 山田太郎
        通貨: ユーロ
    """,
},
]

with get_openai_callback() as usage:
    vectorstore = Chroma.from_documents(documents=docs, embedding=cached_embeddings)
    result = vectorstore.similarity_search(qq)

    retriver = vectorstore.as_retriever()
    
    prompt = ChatPromptTemplate.from_messages([
        # ("system", "あなたは保険の設計書について詳しいものです。以下の設計書の読み、答えを教えてください。\n\n {context}"),
        ("human", "{question}")        
    ])

    example_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=prompt,
        examples=examples,
    )

    final_prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは保険の設計書について詳しいものです。以下の設計書の読み、答えを教えてください。\n\n {context}"),
        example_prompt,
        ("human", "{question}")
    ])
   
    chain = {"context": retriver, "question": RunnablePassthrough()} | final_prompt | llm

    result = chain.invoke(qq);
    print(result)
    print(usage)
